# Démo Docker en 10 minutes

Docker permet de créer des containers applicatifs. Comme LXC il est basé sur les CGROUPS, les CAPABILITIES et les NAMESPACES.
 
Il a été créé par des développeurs en suivant des principes objets (on parle d'infrastructure as code, de commit de container, d'instanciation d'images... ). Son but est de faire tourner une infrastucture en microservice. L'unité de base est l'application pas la la machine virtuelle.


## Prise de contact avec Docker


### Un container très simple:

Ce container exécute une commande ping. L'environnement est celui de la distribution busybox. Le container est détruit une fois la commande exécutée (--rm).Un container qui n'a pas de process s'arrête.


In [1]:
docker run --rm -it busybox ping -c4 www.google.fr

PING www.google.fr (216.58.210.227): 56 data bytes
64 bytes from 216.58.210.227: seq=0 ttl=37 time=0.132 ms
64 bytes from 216.58.210.227: seq=1 ttl=37 time=0.443 ms
64 bytes from 216.58.210.227: seq=2 ttl=37 time=0.434 ms
64 bytes from 216.58.210.227: seq=3 ttl=37 time=0.637 ms

--- www.google.fr ping statistics ---
4 packets transmitted, 4 packets received, 0% packet loss
round-trip min/avg/max = 0.132/0.411/0.637 ms


### Lancement d'un container debian "deb1"



In [14]:
docker run -d --name deb1 --hostname deb1 debian /bin/sh -c 'tailf /dev/null'  

ded901511022877f270e99b9ff31562c1a4ecb25018b6c1b8c8b4f89eb781836


On peut remarquer que Docker génère un  id unique que l'on voit ci-dessus:
La boucle est infinie et tant que ce process tourne le container est vivant.On peut néanmoins lancer d'autre process dans le container:


In [5]:
docker exec -it deb1 bash -c 'ip a'

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: sit0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1
    link/sit 0.0.0.0 brd 0.0.0.0
3: ip6tnl0@NONE: <NOARP> mtu 1452 qdisc noop state DOWN group default qlen 1
    link/tunnel6 :: brd ::
4: ip6gre0@NONE: <NOARP> mtu 1448 qdisc noop state DOWN group default qlen 1
    link/gre6 00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00 brd 00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00
124: eth0@if125: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:11:00:02 brd ff:ff:ff:ff:ff:ff
    inet 172.17.0.2/16 scope global eth0
       valid_lft forever preferred_lft forever
    inet6 fe80::42:acff:fe11:2/64 scope link 
     

In [8]:
docker exec -it deb1 bash -c 'cat /etc/debian_version'

8.4


La commande suivante liste les container actifs. 

In [10]:
docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
2bde6c5bb734        debian              "/bin/sh -c 'tailf /d"   6 minutes ago       Up 6 minutes                            deb1


L'option -a liste les container actifs et inactifs:

In [1]:
docker ps -a

CONTAINER ID        IMAGE                                COMMAND                  CREATED             STATUS                      PORTS                    NAMES
bcaab4c7525c        flaskapp                             "python app.py"          4 minutes ago       Up 4 minutes                0.0.0.0:5000->5000/tcp   backstabbing_murdock
c7d62d5751a0        flaskapp                             "python2"                16 minutes ago      Exited (0) 16 minutes ago                            elegant_banach
fc0cf66b64cb        flaskapp                             "python2"                16 minutes ago      Exited (0) 16 minutes ago                            gloomy_stonebraker
ded901511022        debian                               "/bin/sh -c 'tailf /d"   2 hours ago         Exited (137) 2 hours ago                             deb1
d5a28d49035e        hello-world                          "/hello"                 3 hours ago         Exited (0) 3 hours ago                               naug

On va stopper notre container et le supprimer :

In [2]:
docker stop deb1 && docker rm deb1

deb1
deb1


On vérifie

In [ ]:
docker ps

## Les images
 

Pour créer un container on s'appuie sur des images. Une image permet d'instancier un container. La commande suivante permet de chercher une image centos  sur le registry public de Docker. 
 
 

In [3]:
docker search centos

NAME                            DESCRIPTION                                     STARS     OFFICIAL   AUTOMATED
centos                          The official build of CentOS.                   2237      [OK]       
jdeathe/centos-ssh              CentOS-6 6.7 x86_64 / CentOS-7 7.2.1511 x8...   22                   [OK]
jdeathe/centos-ssh-apache-php   CentOS-6 6.7 x86_64 / Apache / PHP / PHP M...   17                   [OK]
million12/centos-supervisor     Base CentOS-7 with supervisord launcher, h...   11                   [OK]
nimmis/java-centos              This is docker images of CentOS 7 with dif...   10                   [OK]
torusware/speedus-centos        Always updated official CentOS docker imag...   8                    [OK]
nickistre/centos-lamp           LAMP on centos setup                            4                    [OK]
centos/mariadb55-centos7                                                        3                    [OK]
nathonfowlie/centos-jre         Latest CentOS

Il y a surement une image docker pour vous sur le registry Docker ou vous pouvez la générer via un Dockerfile. Ce Dockerfile se contruit à partir d'une image de base officielle Python.
```docker
FROM python:2.7
MAINTAINER jmp "jean-marc.pouchoulon@iutbeziers.fr"
RUN mkdir /code
WORKDIR /code
ADD app.py /code/
ADD requirements.txt /code/
RUN pip install -r requirements.txt
EXPOSE 5000
ENTRYPOINT ["python"]
CMD ["app.py"]
```
La commande suivante génère une image dont le nom est jmp/monappli. Le "." représente le fichier Dockerfile


In [12]:
docker build -t jmp/monappli .


Step 1 : FROM python:2.7
 ---> 11a8b7c7f0ca
Step 2 : MAINTAINER jmp "jean-marc.pouchoulon@iutbeziers.fr"
 ---> Using cache
 ---> 582799d14b4d
Step 3 : RUN mkdir /code
 ---> Using cache
 ---> 583b0877d5b0
Step 4 : WORKDIR /code
 ---> Using cache
 ---> 1db31500eed0
Step 5 : ADD app.py /code/
 ---> Using cache
 ---> 27918081be3c
Step 6 : ADD requirements.txt /code/
 ---> Using cache
 ---> af2f64764eb7
Step 7 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> eec53fab8f81
Step 8 : EXPOSE 5000
 ---> Using cache
 ---> acd901255d96
Successfully built acd901255d96


On vérifie que l'image a été générée:

In [13]:
docker images jmp/*

REPOSITORY           TAG                 IMAGE ID            CREATED             SIZE
jmp/monappli         latest              acd901255d96        22 minutes ago      671.4 MB
jmp/openvpnclient2   latest              cf2bfbc385b4        8 days ago          184 MB
jmp/ovpnserv2        latest              d5575f090621        8 days ago          201.3 MB
jmp/ovpnserv         latest              271dec47207c        8 days ago          184 MB
jmp/debiancurl       latest              21358dd1930d        2 weeks ago         258.8 MB
jmp/debianiut        latest              1a6a371199b7        5 weeks ago         164.1 MB


On peut alors instancier le container en utilisant l'image que l'on vient de contruire et visualiser le "hello world de notre application"

In [15]:
docker run -d -p 5000:5000 jmp/monappli python app.py

cd9950c541c48231b30e607548ff89342a055c5cdbcb2226532427fb0d598135


In [16]:
wget -qO- http://localhost:5000

Hello World!